In [34]:
import numpy as np
import pandas as pd
import requests

from plotnine import *

# source: https://gist.github.com/rogerallen/1583593
us_state_abbrev = requests.get('https://raw.githubusercontent.com/jwhendy/covid19/master/lib/state_abbrevs.json').json()
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

pop = pd.read_excel('https://www2.census.gov/library/publications/2011/compendia/usa-counties/excel/POP01.xls')
pop = pop[['Area_name', 'STCOU', 'POP010210D']]
land = pd.read_excel('https://www2.census.gov/library/publications/2011/compendia/usa-counties/excel/LND01.xls')
land = land[['Areaname', 'STCOU', 'LND110210D']]
age = pd.read_excel('https://www2.census.gov/library/publications/2011/compendia/usa-counties/excel/AGE01.xls',
                    sheet_name='Sheet2')
age = age[['Areaname', 'STCOU', 'AGE050210D']]
inc = pd.read_excel('https://www2.census.gov/library/publications/2011/compendia/usa-counties/excel/INC01.xls')
inc = inc[['Area_name', 'STCOU', 'INC110209D']]

In [64]:
age.head()

,Areaname,STCOU,AGE050210D
0,UNITED STATES,0,37.2
1,ALABAMA,1000,37.9
2,"Autauga, AL",1001,37.0
3,"Baldwin, AL",1003,41.1
4,"Barbour, AL",1005,39.0


In [49]:
### population data: saves out state, county, fips, and population
df = pop.copy()
df.columns = ['area', 'fips', 'pop']
df['state'] = df['area'].str.split(', ', expand=True)[1]
df['county'] = df['area'].str.split(', ', expand=True)[0]
df = df[['state', 'county', 'fips', 'pop']]
df['state'] = df['state'].map(state_abbrevs.abbrev_us_state)
df = df[-df.state.isna()]
df.to_csv('data/population.csv', index=False)
df.head()

,state,county,fips,pop
2,Alabama,Autauga,1001,54571
3,Alabama,Baldwin,1003,182265
4,Alabama,Barbour,1005,27457
5,Alabama,Bibb,1007,22915
6,Alabama,Blount,1009,57322


In [50]:
### land area: saves out state, county, fips, and land in square miles
df = land.copy()
df.columns = ['area', 'fips', 'land_sqm']
df['state'] = df['area'].str.split(', ', expand=True)[1]
df['county'] = df['area'].str.split(', ', expand=True)[0]
df = df[['state', 'county', 'fips', 'land_sqm']]
df['state'] = df['state'].map(state_abbrevs.abbrev_us_state)
df = df[-df.state.isna()]
df.to_csv('data/land_sqm.csv', index=False)
df.head()

,state,county,fips,land_sqm
2,Alabama,Autauga,1001,594.44
3,Alabama,Baldwin,1003,1589.78
4,Alabama,Barbour,1005,884.88
5,Alabama,Bibb,1007,622.58
6,Alabama,Blount,1009,644.78


In [62]:
### age: saves out state, county, fips, and median age
df = age.copy()
df.columns = ['area', 'fips', 'age_med']
df['state'] = df['area'].str.split(', ', expand=True)[1]
df['county'] = df['area'].str.split(', ', expand=True)[0]
df = df[['state', 'county', 'fips', 'age_med']]
df['state'] = df['state'].map(state_abbrevs.abbrev_us_state)
df = df[-df.state.isna()]
df.to_csv('data/age_median.csv', index=False)
df.head()

,state,county,fips,age_med
2,Alabama,Autauga,1001,37.0
3,Alabama,Baldwin,1003,41.1
4,Alabama,Barbour,1005,39.0
5,Alabama,Bibb,1007,37.8
6,Alabama,Blount,1009,39.0


In [63]:
### income: saves out state, county, fips, and median income 2005-2009
df = inc.copy()
df.columns = ['area', 'fips', 'inc_med']
df['state'] = df['area'].str.split(', ', expand=True)[1]
df['county'] = df['area'].str.split(', ', expand=True)[0]
df = df[['state', 'county', 'fips', 'inc_med']]
df['state'] = df['state'].map(state_abbrevs.abbrev_us_state)
df = df[-df.state.isna()]
df.to_csv('data/income_median.csv', index=False)
df.head()

In [138]:
### pulls all mobility data from google
# - google: https://www.google.com/covid19/mobility/
# - quasi api used below: https://github.com/datasciencecampus/mobility-report-data-extractor
import datetime
import pandas as pd
import os
import re
import subprocess
import time

seg_list = [x for _ in range(2)
            for x in ['Retail & recreation', 'Grocery & pharmacy', 'Parks',
                      'Transit stations', 'Workplace', 'Residential']]
path = '/home/jwhendy/vault/personal/covid19/'
dir_mob = 'mobility-report-data-extractor'
### run to re-download and process reports
subprocess.call(['./lib/mobility-script.sh'])
areas = [d for d in os.listdir(os.path.join(path, dir_mob, 'output'))
         if d.startswith('US')]

start = time.time()
data_all = []
for area in areas:
    ### re-process pdfs to text
    f = os.path.join(path, dir_mob, 'pdfs', area)
    subprocess.call(['/usr/bin/pdftotext', '-layout', '-raw', f'{f}.pdf', f'{f}.txt'])
    with open(f'{f}.txt') as f:
        lines = [l for l in f.read().split('\n') if l.strip()]
    #print(lines)
    
    header = re.split(', | ', lines[1])
    date = f'{header[-1]}-{header[-3]}-{header[-2]}'
    date = datetime.datetime.strptime(date, '%Y-%B-%d').strftime('%Y-%m-%d')
    area = ' '.join(header[:-3])
    
    data = []
    for i, line in enumerate(lines):
        if re.findall('Retail & recreation', line) and i<20:
            vals = [re.sub('%|\+', '', lines[i+x]) for x in [1, 13, 26, 38, 49, 59]]
            rows = [{'area': area, 'loc': 'summary', 'seg': seg_list[i], 'conf': None, 'value': vals[i]} for i in range(6)]
            data.extend(rows)
        if re.findall('\f', line) and i>50:
            locs = [x.strip() for x in [lines[i], lines[i+13]] for _ in range(6)]
            locs = [l for l in locs if len(l.split(' ')) < 4]
            asts = [lines[i+n-1] for n, x in enumerate(lines[i:i+110]) if x.startswith('Sun')]
            asts = [0 if ast=='*' else 1 for ast in asts]
            vals = [re.sub('%|\+|compared to baseline', '', lines[i+x])
                    for x in [2, 4, 6, 8, 10, 12, 15, 17, 19, 21, 23, 25]]
            vals = [val.strip(' ') if val != 'Not enough data for this date' else None for val in vals]
            segs = [lines[i+n+1] for n in [0, 2, 4, 6, 8, 10, 13, 15, 17, 19, 21, 23]]
            for i, loc in enumerate(locs):
                if segs[i] not in seg_list:
                    continue
                data.append({'area': area, 'loc': locs[i], 'seg': segs[i], 'conf': asts[i], 'value': vals[i]})

    for i, d in enumerate(data):
        seq = (6*int(i/6))+(i%6)+1
        data[i]['i'] = seq
        data[i]['path'] = f"output/US-{d['area'].replace(' ', '_', -1)}/csv/{seq}.csv"
    
    data_all.extend(data)
end = time.time()
print(end-start)

df = pd.DataFrame(data_all)
df['loc'] = df['loc'].str.split(' County', expand=True)[0]
df['value'] = pd.to_numeric(df['value'])
df['conf'] = pd.to_numeric(df['conf'])

8.855668067932129


In [135]:
df_save = df.copy()
df_save = df_save[df_save['loc'] != 'summary']
df_save = df_save[['area', 'loc', 'seg', 'conf', 'value']]
df_save.columns = ['state', 'county', 'seg', 'conf', 'value']
#df_save['type'] = df_save['type'].map({'Retail & recreation': 'retail_rec',
#                                       'Grocery & pharmacy': 'groc_pharm',
#                                       'Parks': 'parks',
#                                       'Transit stations': 'transit',
#                                       'Workplace': 'work',
#                                       'Residential': 'home'})
df_save.head()
df_save.to_csv('data/mobility-data-agg.csv', index=False)

In [141]:
df.iloc[-100:]

,area,loc,seg,conf,value,i,path
17312,United States,North Dakota,Parks,1.0,73.0,213,output/US-United_States/csv/213.csv
17313,United States,North Dakota,Transit stations,1.0,-36.0,214,output/US-United_States/csv/214.csv
17314,United States,North Dakota,Workplace,1.0,-24.0,215,output/US-United_States/csv/215.csv
17315,United States,North Dakota,Residential,1.0,9.0,216,output/US-United_States/csv/216.csv
17316,United States,Ohio,Retail & recreation,1.0,-43.0,217,output/US-United_States/csv/217.csv
...,...,...,...,...,...,...,...
17407,United States,Wyoming,Grocery & pharmacy,1.0,-13.0,308,output/US-United_States/csv/308.csv
17408,United States,Wyoming,Parks,1.0,29.0,309,output/US-United_States/csv/309.csv
17409,United States,Wyoming,Transit stations,1.0,-14.0,310,output/US-United_States/csv/310.csv
17410,United States,Wyoming,Workplace,1.0,-29.0,311,output/US-United_States/csv/311.csv


In [142]:
ts_all = []
df2 = df.copy()
df2 = df2[df2['seg'] != 'summary']
df2 = df2[df2['area'] != 'United States']

for i, row in df2.iterrows():
    ts = pd.read_csv(os.path.join(path, dir_mob, row.path))[['value', 'date']]
    ts['seg'] = [row.seg]*len(ts)
    ts['state'] = [row.area]*len(ts)
    ts['county'] = [row['loc']]*len(ts)
    ts_all.append(ts)

In [145]:
df_ts = pd.concat(ts_all)
df_ts = df_ts[['state', 'county', 'seg', 'date', 'value']]
df_ts

In [146]:
df_ts.to_csv('data/mobility-data-ts.csv', index=False)